# Process All Images

#### Imports

In [ ]:
%pylab inline
%matplotlib inline

import numpy as np
import cv2
import time
import pickle
import ImageProcessing as ipr
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display

#### Load the images

In [ ]:
allData = []

#numPictures = input('number of pictures taken ')*3
numPictures = 12*3


for i in range(36):
    trainingFileName = 'my data/png data/image_png_' + str(i+1) + '.png'
    trainingPngFile = imread(trainingFileName)
    trainingFile = trainingPngFile
    allData.append(trainingFile)
    
allImages = ipr.from_float_to_uint8(allData)    
    
print ('Succesfully loaded ' + str(len(allImages)) + ' images!')

##### Display all the images

In [ ]:
limit = 115
fig = figure (0, (8, len(allImages)))

for i in range(len(allImages)):
    subplot(len(allImages)/3, 3, i+1)
    if i%3 == 0:
        ylabel('Image n°' + str(round(i/3)+1), size='xx-large')
        imshow(allImages[i])
    if i%3 == 1:
        imshow(allImages[i], cmap='gray')
    if i%3 == 2:
        imshow(allImages[i]>limit, cmap='gray')

#### Keep only wanted images

In [ ]:
trainingImages = []

for i in range(15):
    trainingImages.append(allImages[i])
for i in range(18, 21):
    trainingImages.append(allImages[i])
for i in range(24, 30):
    trainingImages.append(allImages[i])
for i in range(33, 36):
    trainingImages.append(allImages[i])

print ('Succesfully loaded ' + str(round(len(trainingImages)/3))\
       + ' triaining images!')

#### Display final set

In [ ]:
fig = figure (0, (8, len(trainingImages)))

for i in range(len(trainingImages)):
    subplot(len(trainingImages)/3, 3, i+1)
    if i%3 == 0:
        ylabel('Image n°' + str(round(i/3)+1), size='xx-large')
        imshow(trainingImages[i])
    if i%3 == 1:
        imshow(trainingImages[i], cmap='gray')
    if i%3 == 2:
        imshow(trainingImages[i]>limit, cmap='gray')

#### Create and display binary and black & white images

In [ ]:
boolImages = []
bnImages = []

for i in range(len(trainingImages)):
    if i%3 == 2:
        boolImages.append(trainingImages[i]>limit)

for i in range(len(boolImages)):
    image = np.zeros((len(boolImages[i]), len(boolImages[i][0]), 3), dtype='uint8')
    for x in range(len(boolImages[i])):
        for y in range(len(boolImages[i][x])):
            if boolImages[i][x, y] == True:
                image[x, y] = [255, 255, 255]
            elif boolImages[i][x, y] == False:
                image[x, y] = [0, 0, 0]
    bnImages.append(image)
    
print('Successfuly created '+ str(len(boolImages)) + ' images !')

In [ ]:
fig = figure(0, (15, 4))
for i in range(len(boolImages)):
    subplot(2, len(boolImages), i+1)
    title('Image n°'+ str(i+1))
    axis('off')
    imshow(boolImages[i])
    subplot(2, len(boolImages), i+1+len(boolImages))
    axis('off')
    imshow(bnImages[i])

#### Save black & white images to labelize

In [ ]:
#TO SAVE GREY SCALE PICTURES IN ORDER TO CREATE LABELIZED IMAGES
#
#for i in range(len(bnImages)):
#    im = Image.fromarray(bnImages[i])
#    im.save('my data/png data/training_label_' + str(i+1) +'.png')

#### Load and display labelized images

In [ ]:
floatLabel = []
for i in range(9):
    trainingFileName = 'my data/png data/training_label_' + str(i+1) +'.png'
    trainingPngFile = imread(trainingFileName)
    trainingFile = trainingPngFile
    floatLabel.append(trainingFile)
    
labelImages = ipr.from_float_to_uint8(floatLabel)

print ('Succesfully loaded ' + str(len(labelImages)) + ' labelized images!')

In [ ]:
fig = figure(0, (20, 10))

rows = 3
cols = len(labelImages)

for i in range(cols):
    subplot(rows, cols, i+1)
    title('Image n°' + str(i+1))
    imshow(trainingImages[(i*3+2)], cmap='gray')
    subplot(rows, cols, i+1+cols)
    imshow(bnImages[i], cmap='gray')
    subplot(rows, cols, i+1+cols*2)
    imshow(labelImages[i], cmap='gray')

#### Create and display a list of directories

In [ ]:
finalTrainingDict = []

for i in range(len(trainingImages)):
    if i%3 == 0:
        image = {}
        image['image'] = trainingImages[i]
        image['cropped'] = trainingImages[i+1]
        image['resized'] = trainingImages[i+2]
        image['bool'] = boolImages[round(i/3)]
        image['label'] = labelImages[round(i/3)]
        image['blacknwhite'] = bnImages[round(i/3)]
        finalTrainingDict.append(image)

print('Dictionary successfuly completed with ' + \
      str(len(finalTrainingDict))+ ' images !')

In [ ]:
fig = figure(0, (20, 36))

cols = len(finalTrainingDict[0].keys())
rows = len(labelImages)

for i in range(rows):
    subplot(rows, cols, i*cols+1)
    title('Image n°' + str(i+1))
    imshow(finalTrainingDict[i]['image'], cmap='gray')
    subplot(rows, cols, i*cols+2)
    imshow(finalTrainingDict[i]['cropped'], cmap='gray')
    subplot(rows, cols, i*cols+3)
    imshow(finalTrainingDict[i]['resized'], cmap='gray')
    subplot(rows, cols, i*cols+4)
    imshow(finalTrainingDict[i]['blacknwhite'], cmap='gray')
    subplot(rows, cols, i*cols+5)
    imshow(finalTrainingDict[i]['bool'], cmap='gray')
    subplot(rows, cols, i*cols+6)
    imshow(finalTrainingDict[i]['label'], cmap='gray')

#### Create binaries libelized images

In [ ]:
dist = 4
boxStart = dist

for i in range(len(finalTrainingDict)):
    boxXEnd = len(finalTrainingDict[i]['label']) - boxStart
    boxYEnd = len(finalTrainingDict[i]['label'][0]) - boxStart
    #print(boxStart, boxXEnd, boxYEnd)
    
    actual = np.full((boxXEnd - boxStart, boxYEnd - boxStart), False, dtype='bool')
    #predicted = np.zeros((boxXEnd - boxStart, boxYEnd - boxStart, 3), dtype='uint8')
    for x in range(boxStart, boxXEnd):
        for y in range(boxStart, boxYEnd):
            rValue = finalTrainingDict[i]['label'][x, y][0]
            gValue = finalTrainingDict[i]['label'][x, y][1]
            bValue = finalTrainingDict[i]['label'][x, y][2]
            rgbValue = [rValue, gValue, bValue]
            if rgbValue == [0, 0, 255]:
                actual[x - boxStart, y - boxStart] = True
                
    finalTrainingDict[i]['predicted'] = \
    np.zeros((boxXEnd - boxStart, boxYEnd - boxStart, 3), dtype='uint8')
    finalTrainingDict[i]['actual'] = actual
    finalTrainingDict[i]['fingerPoints'] = sum(actual == True)

print(str(i+1) + ' images were correctly labelized')

In [ ]:
fig = figure(0, (15, 4))
for i in range(len(finalTrainingDict)):
    subplot(2, len(finalTrainingDict), i+1)
    
    lengthX = len(finalTrainingDict[i]['label'])
    lengthY = len(finalTrainingDict[i]['label'][0])

    imshow(finalTrainingDict[i]['label'][4: lengthX - 4,4: lengthY - 4], cmap='gray')
    subplot(2, len(finalTrainingDict), i+1+len(finalTrainingDict))
    imshow(finalTrainingDict[i]['actual'], cmap='gray')

#### Create all convolutions

In [ ]:
for index in range(len(finalTrainingDict)):
    boxXMax = len(finalTrainingDict[index]['bool']) - 4
    boxYMax = len(finalTrainingDict[index]['bool'][0]) - 4
    allBoxs = []
    for i in range(4, boxXMax):
        for j in range(4, boxYMax):
            box = finalTrainingDict[index]['bool'][i - 4:i + 5, j - 4:j + 5]
            allBoxs.append(box)
    finalTrainingDict[index]['boxs'] = allBoxs
        
print(str(len(finalTrainingDict[0]['boxs'])*len(finalTrainingDict)) +\
      ' boxes were created from '+ \
          str(len(finalTrainingDict))+' training !')

#### Display convolution examples

In [ ]:
fig = figure(0, (20, 10))
for i in range(len(finalTrainingDict)):
    subplot(1, len(finalTrainingDict), i+1)
    imshow(finalTrainingDict[i]['boxs'][419], cmap='gray')

#### Dump the treated data

In [ ]:
#pickleFile = open('my data/'+ 'treatedData' + '.pkl', 'wb')
#pickle.dump(finalTrainingDict, pickleFile, pickle.HIGHEST_PROTOCOL)
#pickleFile.close()